# Cvičení 2: Drift trosek na hladině

**Motivace:** Lehké těleso (troska, bójka, kus plastu) na hladině je unášeno okolním prostředím – zejména větrem a proudem. Chceme numericky spočítat jeho trajektorii v rovině.

V tomto cvičení budeme používat **zjednodušený model**:
- pohyb je jen ve 2D rovině (x–y),
- prostředí má v každém bodě a čase rychlost **vektorového pole** $\mathbf{w}(\mathbf{r}, t)$,
- jediná síla je odpor proti **relativní rychlosti** $\mathbf{v}-\mathbf{w}$,
- detaily jako vlny, rotace tělesa, změna ponoření apod. ignorujeme.

Cílem je pochopit vazbu „rychlost → poloha“ a „síla → zrychlení → rychlost“ a umět postavit jednoduchou simulaci.

**Cíle:**
- formálně popsat polohu $\mathbf{r}(t)=(x(t),y(t))$, rychlost $\mathbf{v}(t)$ a zrychlení $\mathbf{a}(t)$ ve 2D,
- vysvětlit vztah mezi silou, zrychlením a hmotností (Newtonův 2. zákon),
- chápat rychlost prostředí jako vektorové pole $\mathbf{w}(\mathbf{r}, t)$ $[\mathrm{m}\,\mathrm{s}^{-1}]$,
- převést relativní rychlost $\mathbf{v}-\mathbf{w}$ na odporovou sílu,
- pracovat s parametrem $\kappa = \alpha/m$ $[\mathrm{m}^{-1}]$ charakterizujícím poměr odporu a hmotnosti unášeného tělesa jakožto jediným volným parametrem rovnice pohybu,
- použít Eulerovu metodu pro modelování vztahu mezi rychlostí a zrychlením,
- a lichoběžníkové pravidlo pro výpočet polohy z rychlosti.

**Zadání (shrnutí):** Částice o hmotnosti $m$ a koeficientu odporu $\alpha$ startuje v bodě $(x_0, y_0)$ v čase $t_0$ s nulovou počáteční rychlostí $\mathbf{v}_0=(0,0)$. Pohybuje se v prostředí s rychlostí $\mathbf{w}(\mathbf{r}, t)$. Vypočtěte trajektorii do času $t_1$.


---


## 1. Okamžitá poloha a okamžitá rychlost

**Vektor polohy:** Stav částice v čase $t$ popisujeme vektorem polohy v rovině
$$
\mathbf{r}(t) = (x(t), y(t)).
$$
Jednotkou je metr $[\mathrm{m}]$.

**Rychlost a zrychlení:** Okamžitá rychlost a zrychlení jsou derivace podle času:
$$
\mathbf{v}(t) = \mathbf{r}'(t), \qquad \mathbf{a}(t) = \mathbf{v}'(t).
$$
- $\mathbf{v}(t)$ říká, **kam** a **jak rychle** se částice pohybuje $[\mathrm{m}\,\mathrm{s}^{-1}]$.
- $\mathbf{a}(t)$ říká, jak se rychlost mění (zrychluje / zpomaluje / stáčí) $[\mathrm{m}\,\mathrm{s}^{-2}]$.

**Doplňující kontext:** Obrázky níže ukazují jednoduchý 2D pohyb. Trajektorie v rovině x–y, a časové průběhy $x(t)$, $y(t)$, $v_x(t)$, $v_y(t)$, $a_x(t)$, $a_y(t)$.

<img src="images/kinematika_trajektorie.png" alt="Trajektorie v rovině x–y" width="480"/>

<table>
  <tr>
    <td><img src="images/kinematika_x_t.png" alt="x(t)" width="320"/></td>
    <td><img src="images/kinematika_y_t.png" alt="y(t)" width="320"/></td>
  </tr>
  <tr>
    <td><img src="images/kinematika_vx_t.png" alt="v_x(t)" width="320"/></td>
    <td><img src="images/kinematika_vy_t.png" alt="v_y(t)" width="320"/></td>
  </tr>
  <tr>
    <td><img src="images/kinematika_ax_t.png" alt="a_x(t)" width="320"/></td>
    <td><img src="images/kinematika_ay_t.png" alt="a_y(t)" width="320"/></td>
  </tr>
</table>


---


## 2. Numerický výpočet trajektorie ze znalosti rychlosti

Známe-li rychlost $\mathbf{v}(t)$, polohu dostaneme pomocí integrace:
$$
\mathbf{r}(t)=\mathbf{r}_0+\int_{t_0}^{t}\mathbf{v}(\tau)\,d\tau.
$$
V praxi ale rychlost neznáme jako „hezký vzorec“, spíš ji máme spočítanou v diskrétních časových bodech. Proto integrál nahradíme numerickým součtem.

**Diskrétní čas:** Zavedeme časové body
$$
t_i=t_0+i\,\Delta t, \qquad i=0,1,\dots,N.
$$
Rychlost známe v těchto bodech: $\mathbf{v}_i=\mathbf{v}(t_i)$. Cílem je spočítat polohy $\mathbf{r}_i=\mathbf{r}(t_i)$.

**Lichoběžníkové pravidlo:** V jednom kroku aproximujeme plochu pod grafem rychlosti (v 1D) lichoběžníkem.
Pro složku $x$ dostaneme
$$
x_i=x_{i-1}+\frac{v_{x,i-1}+v_{x,i}}{2}\,\Delta t.
$$
Stejně pro složku $y$.

Prakticky to znamená:
1. připravit pole časů `t`,
2. mít rychlosti `v[i]` ve stejných časech,
3. jít po krocích a počítat `r[i]` z `r[i-1]`.

![Lichoběžníkové pravidlo](images/lichobeznikove_pravidlo.png)


---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


### Pomocná funkce `vykresli_drift(t, r, v=None, a=None, title=None)`

**K čemu je:** Jednotné vykreslení výsledků simulace. První obrázek je trajektorie v rovině (x–y) se startem a koncem. Pokud je zadáno `v`, druhý obrázek je mřížka 2×2 s průběhy $x(t), y(t), v_x(t), v_y(t)$. Pokud je zadáno i `a`, přibude řádek s $a_x(t), a_y(t)$. Bez `v` ani `a` se vykreslí jen $x(t), y(t)$.

**Vstupy:**
- `t`: 1D pole časů $[\mathrm{s}]$
- `r`: pole tvaru (n, 2) s polohou $[\mathrm{m}]$
- `v`: pole tvaru (n, 2) s rychlostí $[\mathrm{m}\,\mathrm{s}^{-1}]$ (volitelné)
- `a`: pole tvaru (n, 2) se zrychlením $[\mathrm{m}\,\mathrm{s}^{-2}]$ (volitelné)
- `title`: nadpis grafu trajektorie (volitelné)


In [ ]:
from cv2_utils import vykresli_drift


---
## Úkol 1: Trajektorie z dané rychlosti

**Zadání:** Uvažujte částici se startem $\mathbf{r}_0 = (0, 0)$ a časově proměnnou rychlostí
$$
\mathbf{v}(t) = (1 + 0.5\cos(0.5 t),\; 0.3\sin(0.5 t))\,\mathrm{m}\,\mathrm{s}^{-1}.
$$
Pro časový krok $\Delta t = 0{.}1\,\mathrm{s}$ a interval
$t \in [0, 10]\,\mathrm{s}$ spočtěte trajektorii v časových bodech $t_i = t_0 + i\,\Delta t$.

**Požadavky na řešení:**
1. vytvořte pole časů `t` včetně koncového času,
2. spočtěte pole rychlostí `v` podle předpisu a prealokujte `r` ve tvaru `(n, 2)`,
3. implementujte lichoběžníkové pravidlo v cyklu,
4. zkontrolujte výsledek pomocí analytického řešení:
   $$\mathbf{r}(t) = (t + \sin(0.5 t),\; 0.6(1-\cos(0.5 t))).$$


In [ ]:
def poloha_2d(r0, v, dt):
    """
    Numericky integruje polohu z rychlosti lichoběžníkovým pravidlem.

    Parametry:
        r0 : počáteční poloha [m], pole tvaru (2,)
        v  : rychlost v čase, pole tvaru (n, 2) [m/s]
        dt : krok času [s]

    Návrat:
        r : poloha v čase, pole tvaru (n, 2) [m]
    """
    ###### doplňte kód zde ######

    v = np.asarray(v, dtype=float)
    n = v.shape[0]
    r = np.zeros((n, 2), dtype=float)
    r[0] = np.asarray(r0, dtype=float)
    for i in range(1, n):
        # lichoběžník: průměr rychlostí na kraji intervalu krát délka intervalu
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt

    #############################
    return r


# Zadání pro test
###### doplňte kód zde ######
r0 = np.array([0.0, 0.0])
T = 10.0
dt = 0.1
t = np.arange(0.0, T + dt, dt)
v_prescribed = np.column_stack(
    (
        1.0 + 0.5 * np.cos(0.5 * t),
        0.3 * np.sin(0.5 * t),
    )
)
#############################

# Výpočet polohy z rychlosti
r = poloha_2d(r0, v_prescribed, dt)

# Kontrola: pro předepsanou rychlost známe přesné řešení
r_exact = np.column_stack(
    (
        r0[0] + t + np.sin(0.5 * t),
        r0[1] + 0.6 * (1 - np.cos(0.5 * t)),
    )
)
max_err = np.max(np.linalg.norm(r - r_exact, axis=1))
print(f"Koncová poloha r(T) = {r[-1]} m (očekáváme {r_exact[-1]})")
print(f"Max. chyba oproti přesnému řešení: {max_err:.3e} m")

vykresli_drift(t, r, v_prescribed, title="Úkol 1: trajektorie z předepsané rychlosti")


---


## 3. Numerický výpočet rychlosti ze zrychlení – Eulerova metoda

Známe-li zrychlení jako **diskrétní hodnoty** v čase: v bodech $t_i$, tedy známe vektor
$\mathbf{a}_i$, Eulerova metoda spočte rychlost po krocích. V každém
intervalu $[t_{i-1}, t_i]$ předpokládá, že zrychlení je konstantní a rovné
$\mathbf{a}_{i-1}$.

![Eulerova metoda](images/eulerova_metoda.png)

Složkově (v rovině):
$$
\begin{aligned}
v_{x,i} &= v_{x,i-1} + a_{x,i-1}\,\Delta t,\\
v_{y,i} &= v_{y,i-1} + a_{y,i-1}\,\Delta t.
\end{aligned}
$$
Vektorově:
$$
\mathbf{v}_i = \mathbf{v}_{i-1} + \mathbf{a}_{i-1}\,\Delta t,
$$
kde $\Delta t = t_i - t_{i-1}$.


Euler je jednoduchý, ale citlivý na velikost $\Delta t$ – příliš velký krok vede
k velkým chybám i nestabilitě.


---


## Úkol 2: Rychlost a poloha z daného zrychlení (diskrétní čas)

**Zadání:** Zrychlení je dané jako pole hodnot $\mathbf{a}_i$ v diskrétních časech
$t_i$ (pole `a` tvaru `(n, 2)`). Zrychlení použijte ve tvaru
$$
\mathbf{a}_i = (0.4\cos(0.2 t_i), 0.3\sin(0.1 t_i))\,\mathrm{m}\,\mathrm{s}^{-2},
$$
kde $t_i = i\,\Delta t$ a $i=0,1,\dots,n$.

Pro kontrolu (pro $\mathbf{v}_0=(0,0)$ a $\mathbf{r}_0=(0,0)$) platí analyticky:
$$
v_x(t) = 2\sin(0.2 t), \qquad v_y(t) = 3\left(1-\cos(0.1 t)\right),
$$
$$
r_x(t) = 10\left(1-\cos(0.2 t)\right), \qquad r_y(t) = 3t - 30\sin(0.1 t).
$$

Z nulové počáteční rychlosti $\mathbf{v}_0=(0,0)$:

1. doplňte Eulerův krok ve funkci `euler_rychlost_2d`,
2. spočtěte rychlost $\mathbf{v}(t)$ z daného pole `a`,
3. z rychlosti spočtěte polohu pomocí `poloha_2d` z Úkolu 1,
4. trajektorii a průběhy (včetně zrychlení) vykreslete pomocí `vykresli_drift`,
5. porovnejte numerické řešení s analytickým a vypište maximální chybu pro rychlost i polohu.


In [ ]:
# Úkol 2 – rychlost a poloha z daného zrychlení
def euler_rychlost_2d(v0, a, dt, T):
    """
    Výpočet rychlosti pomocí Eulerovy metody v rovině.

    Zrychlení je dáno jako pole hodnot v diskrétních časech: a[i] odpovídá t_i = i*dt.

    Parametry:
        v0 : počáteční rychlost [m/s], pole tvaru (2,)
        a  : zrychlení v čase, pole tvaru (n, 2) [m/s^2]
        dt : konstantní krok času [s]
        T  : časový horizont [s] (t0 je pevně 0)

    Návrat:
        v : rychlost v čase, pole tvaru (n, 2) [m/s]
    """
    ###### doplňte kód zde ######
    a = np.asarray(a, dtype=float)
    n = int(np.ceil(T / dt)) + 1
    a = a[:n]

    v = np.zeros((n, 2), dtype=float)
    v[0] = np.asarray(v0, dtype=float)

    for i in range(1, n):
        v[i] = v[i - 1] + a[i - 1] * dt
    #############################

    return v


###### doplňte kód zde ######
v0 = np.array([0.0, 0.0])

T = 40.0
dt = 0.05
t = np.arange(0.0, T + dt, dt)

# zadané zrychlení v diskrétních časech
# a = ...
a = np.zeros((t.size, 2), dtype=float)
a[:, 0] = 0.4 * np.cos(0.2 * t)
a[:, 1] = 0.3 * np.sin(0.1 * t)
#############################

v = euler_rychlost_2d(v0, a, dt, T)

# Poloha z rychlosti (Úkol 1)
r0 = np.array([0.0, 0.0])
r = poloha_2d(r0, v, dt)

# Analytické řešení pro kontrolu
v_exact = np.column_stack((2.0 * np.sin(0.2 * t), 3.0 * (1.0 - np.cos(0.1 * t))))
r_exact = np.column_stack((10.0 * (1.0 - np.cos(0.2 * t)), 3.0 * t - 30.0 * np.sin(0.1 * t)))

v_err = np.linalg.norm(v - v_exact, axis=1)
r_err = np.linalg.norm(r - r_exact, axis=1)
print(f"Max. chyba rychlosti: {v_err.max():.3e} m/s")
print(f"Max. chyba polohy: {r_err.max():.3e} m")

vykresli_drift(t, r, v, a=a, title="Úkol 2: rychlost a poloha z daného zrychlení")


---


## 4. Síla, zrychlení a odpor prostředí

Z Newtonova druhého zákona platí vztah mezi silou, hmotností a zrychlením:
$$
\mathbf{F}(t) = m\,\mathbf{a}(t) \qquad \Rightarrow \qquad \mathbf{a}(t)=\frac{\mathbf{F}(t)}{m}.
$$
Intuice:
- větší síla způsobí větší změnu rychlosti,
- větší hmotnost znamená větší „setrvačnost“ (těžší věc se hůř rozjíždí i brzdí).

V našem modelu působí pouze **odporová síla** (brzdění) proti **relativní** rychlosti vůči prostředí:
$$
\mathbf{v}_\text{rel} = \mathbf{v}(t) - \mathbf{w}(\mathbf{r}, t).
$$
Použijeme tzv. **kvadratický odpor**, kdy velikost síly roste s druhou mocninou rychlosti a směr síly je opačný ke směru pohybu:
$$
\mathbf{F}_\text{d}(t)
= -\alpha\,\|\mathbf{v}_\text{rel}\|^2\,\frac{\mathbf{v}_\text{rel}}{\|\mathbf{v}_\text{rel}\|}
= -\alpha\,\|\mathbf{v}_\text{rel}\|\,\mathbf{v}_\text{rel}.
$$

První tvar explicitně ukazuje rozklad na **velikost** ($\|\mathbf{v}_\text{rel}\|^2$) a **směr** ($\mathbf{v}_\text{rel}/\|\mathbf{v}_\text{rel}\|$).
Záporné znaménko znamená, že odpor působí **proti** relativnímu pohybu.


Dosazením do Newtonova zákona:
$$
\mathbf{a}(t) = \frac{\mathbf{F}_\text{d}(t)}{m}
= -\frac{\alpha}{m}\,\|\mathbf{v}_\text{rel}\|\,\mathbf{v}_\text{rel}
= -\kappa\,\|\mathbf{v}_\text{rel}\|\,\mathbf{v}_\text{rel},
$$
kde
$$
\kappa = \frac{\alpha}{m}
$$
je zjednodušený koeficient odporu.

**Důležité:** Skalár $\|\mathbf{v}_\text{rel}\|$ je jedna společná velikost pro obě složky. Nejde o to, že bychom zvlášť brali $|v_x-w_x|$ a $|v_y-w_y|$.

**Jednotky:** $\mathbf{v}$ a $\mathbf{w}$ $[\mathrm{m}\,\mathrm{s}^{-1}]$, $\mathbf{r}$ $[\mathrm{m}]$, $\mathbf{a}$ $[\mathrm{m}\,\mathrm{s}^{-2}]$, $\alpha$ $[\mathrm{kg}\,\mathrm{m}^{-1}]$, $\kappa$ $[\mathrm{m}^{-1}]$.

**Co očekávat:** Pokud je $\mathbf{w}$ konstantní v čase i prostoru, pak odpor způsobí, že se $\mathbf{v}(t)$ postupně přiblíží k $\mathbf{w}$ (relativní rychlost míří k nule).


### Proč se rychlost přibližuje k rychlosti prostředí?

Když je $\mathbf{w}$ konstantní, pak pro relativní rychlost $\mathbf{v}_\text{rel}=\mathbf{v}-\mathbf{w}$ platí:
- pokud $\mathbf{v}=\mathbf{w}$, pak $\mathbf{v}_\text{rel}=\mathbf{0}$ a odpor je nulový (nic už rychlost nemění),
- pokud $\mathbf{v}\neq\mathbf{w}$, odpor míří proti $\mathbf{v}_\text{rel}$, a tedy relativní rychlost „zmenšuje“.

Proto v typické stabilní simulaci uvidíte, že $\|\mathbf{v}(t)-\mathbf{w}\|$ s časem klesá.


Jakmile je zrychlení funkcí rychlosti a (případně) polohy, nejde nejdřív spočítat celé pole $\mathbf{a}$, pak celé $\mathbf{v}$ a nakonec celé $\mathbf{r}$. Musíme postupovat po krocích: z aktuálních $\mathbf{v}$ a $\mathbf{r}$ spočteme $\mathbf{a}$, z $\mathbf{a}$ aktualizujeme $\mathbf{v}$ (Euler) a z $\mathbf{v}$ pak $\mathbf{r}$ (lichoběžník). Proto v dalším úkolu připravíme obecnou simulační funkci.

---


## Úkol 3: Obecná simulace pohybu + test driftu

**Zadání:** Napište funkci `simuluj_drift`, která pro zadané `r0`, `v0`, `dt`, `T`
a funkci `a_funkce(v, r, t)` vrátí trojici polí `a`, `v`, `r` (vždy tvaru `(n, 2)`).

**Požadavky na řešení:**
1. použijte Eulerův krok pro rychlost,
2. použijte lichoběžníkové pravidlo pro polohu,
3. zrychlení počítejte v každém kroku z aktuálního stavu.

**Otestujte funkčnost:** Pro parametry v kódu (start $\mathbf{r}_0$, $\mathbf{v}_0$, $\kappa$, $\mathbf{w}$,
$T$, $\Delta t$) spusťte simulaci driftu a ověřte, že se rychlost částice přibližuje k $\mathbf{w}$.

Funkci využijete i v následujících úlohách.


In [ ]:
def simuluj_drift(r0, v0, dt, T, a_funkce):
    """
    Simulace pohybu: Euler pro rychlost a lichoběžník pro polohu.

    Parametry:
        r0      : počáteční poloha [m], pole tvaru (2,)
        v0      : počáteční rychlost [m/s], pole tvaru (2,)
        dt      : konstantní krok času [s]
        T       : časový horizont [s] (t0 je pevně 0)
        a_funkce: funkce a(v, r, t) vracející zrychlení [m/s^2]

    Návrat:
        r : poloha v čase, pole tvaru (n, 2) [m]
        v : rychlost v čase, pole tvaru (n, 2) [m/s]
        a : zrychlení v čase, pole tvaru (n, 2) [m/s^2]
    """
    ###### doplňte kód zde ######
    n = int(np.ceil(T / dt)) + 1

    r = np.zeros((n, 2), dtype=float)
    v = np.zeros((n, 2), dtype=float)
    a = np.zeros((n, 2), dtype=float)

    r[0] = np.asarray(r0, dtype=float)
    v[0] = np.asarray(v0, dtype=float)
    a[0] = a_funkce(v[0], r[0], 0.0)

    for i in range(1, n):
        t_i = i * dt
        v[i] = v[i - 1] + a[i - 1] * dt
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt
        a[i] = a_funkce(v[i], r[i], t_i)
    #############################

    return r, v, a


In [ ]:
# Parametry systému
###### doplňte kód zde ######
r0 = np.array([0.0, 0.0])  # m
v0 = np.array([0.0, 0.0])  # m/s

dt = 0.05  # s
T = 300.0  # s

t = np.arange(0.0, T + dt, dt)

m = 0.001  # kg
alpha = 2.0e-5  # kg/m
kappa = alpha / m  # 1/m

w = np.array([1.2, 0.4])  # m/s
#############################


def a_drift(v, r, t):
    """Zrychlení pro drift s kvadratickým odporem vůči prostředí."""

    ###### doplňte kód zde ######
    v_rel = v - w
    v_rel_norm = np.linalg.norm(v_rel)
    a = -kappa * v_rel_norm * v_rel
    #############################

    return a


r_drift, v_drift, a_drift_hist = simuluj_drift(r0, v0, dt, T, a_drift)
vykresli_drift(t, r_drift, v_drift, a_drift_hist, title="Úkol 3: drift s odporem vůči prostředí")

**Pozorování:** Protože počáteční rychlost je nulová a prostředí má nenulovou rychlost $\mathbf{w}$, částice se nejprve „rozjíždí“ (zrychluje). Odpor ale roste s relativní rychlostí, takže postupně přibrzdí tak, že se rychlost částice přiblíží k rychlosti prostředí.

Pro konstantní $\mathbf{w}$ platí $\mathbf{v}(t) \to \mathbf{w}$, tedy také $\|\mathbf{v}(t)-\mathbf{w}\|\to 0$ a $\|\mathbf{v}(t)\| \to \|\mathbf{w}\|$.


---


## 5. Rychlost prostředí jako časově proměnné vektorové pole

**Hlavní myšlenka:** Rychlost prostředí popíšeme vektorovým polem, které každému bodu a času přiřadí vektor rychlosti $[\mathrm{m}\,\mathrm{s}^{-1}]$:
$$
\mathbf{w}(\mathbf{r}, t) = (w_x(\mathbf{r}, t), w_y(\mathbf{r}, t)).
$$
Nehomogenní pole znamená, že se vektor liší v prostoru. Časově proměnné pole znamená, že se pole mění v čase (např. při průchodu fronty).

**Doplňující kontext:** Představte si oblast o velikosti zhruba $2000\times 2000\,\mathrm{km}$ rozdělenou na mřížku. V každém bodě známe vektor $\mathbf{w}$ a takový „snímek“ se opakuje třeba každý den po dobu 30 dní.

Následující dva satelitní snímky (měsíční průměry) ukazují skutečná data rychlosti větru nad oceánem; barvy reprezentují velikost a šipky směr vektorů rychlosti.

![Měsíční průměrné větry, listopad 2014](images/rapidscat_nov2014.jpg)
*Zdroj: NASA/JPL-Caltech, RapidScat (PIA20365), public domain.*

![Měsíční průměrné větry, listopad 2015](images/rapidscat_nov2015.jpg)
*Zdroj: NASA/JPL-Caltech, RapidScat (PIA20365), public domain.*


---


## Úkol 4: Drift v prostorově proměnném poli

**Zadání:** Uvažujte pole rychlosti prostředí
$\mathbf{w}(\mathbf{r}, t) = k\,(\mathbf{r}^* - \mathbf{r})$ s parametry
$k = 1.5\times 10^{-4}\,\mathrm{s}^{-1}$ a $\mathbf{r}^* = (10000, 5000)\,\mathrm{m}$. Doplňte definici
`w_ext_pole` a simulaci spusťte se stejnými parametry jako v Úkolu 3,
ale s počáteční polohou $\mathbf{r}_0=(10000, 0)\,\mathrm{m}$ a počáteční rychlostí
$\mathbf{v}_0=(100, 0)\,\mathrm{m}\,\mathrm{s}^{-1}$. Trajektorii vykreslete a porovnejte její zakřivení.


In [ ]:
###### doplňte kód zde ######
k = 1.5e-4  # 1/s
r_star = np.array([10_000.0, 5_000.0])
r0_pole = np.array([10_000.0, 0.0])
v0_pole = np.array([100.0, 0.0])
dt = 0.0001  # s
T = 2000.0  # s
t = np.arange(0.0, T + dt, dt)
#############################


def w_ext_pole(r, t):
    """
    Příklad pole w(r, t): rychlost prostředí směřuje k bodu r_star.
    """
    ###### doplňte kód zde ######
    w_loc = k * (r_star - r)
    #############################
    return w_loc


def a_drift_pole(v, r, t):
    v_rel = v - w_ext_pole(r, t)
    v_rel_norm = np.linalg.norm(v_rel)
    return -kappa * v_rel_norm * v_rel


r_pole, v_pole, a_pole = simuluj_drift(r0_pole, v0_pole, dt, T, a_drift_pole)

vykresli_drift(t, r_pole, v_pole, title="Úkol 4: drift v poli rychlosti")


---


## Úkol 5: Trajektorie v časově proměnném vektorovém poli

**Zadání:** Spočtěte trajektorii částice v časově proměnném poli rychlosti.
Doplňte výpočet zrychlení v `a_drift_cas` tak, aby používal `w_ext_grid(r, t)`
a koeficient odporu `kappa_field`, a pak simulaci spusťte a trajektorii vykreslete.


### Pomocné funkce pro časově proměnné pole (`cv2_utils.py`)

**K čemu jsou:** Zabalí generování a vyhodnocení časově proměnného vektorového
pole tak, aby v notebooku zůstala jen podstata úlohy.

**`generate_synthetic_w_grid(L, nx, ny, n_days, day)`**
- vytvoří syntetické pole na mřížce,
- vrací `x` a `y` $[\mathrm{m}]$, `t_grid` $[\mathrm{s}]$ a `W_grid` tvaru `(nt, ny, nx, 2)` v jednotkách $[\mathrm{m}\,\mathrm{s}^{-1}]$.

**`w_synthetic(x, y, t, L, day)`**
- vrátí přesnou hodnotu syntetického pole v bodě (x, y) a čase t,
- používá stejný analytický předpis jako generátor mřížky,
- v simulaci s ním pracujeme místo vzorkování z mřížky.

**`w_from_grid_mesh(X, Y, t, W_grid, x_grid, y_grid, t_grid, interpolate_time=True)`**
- vektorizované vyhodnocení na celé mřížce (užitečné pro vlastní quiver),
- v notebooku ho přímo nepoužíváme, ale je připravené v `cv2_utils.py`.

**`save_vector_field_animation(W_grid, x, y, t_grid, ...)`**
- uloží animaci šipek do mp4,
- vrátí cestu k uloženému souboru (vyžaduje `ffmpeg`),
- pro plynulost používá lineární interpolaci v čase.


In [ ]:
from cv2_utils import (
    generate_synthetic_w_grid,
    w_synthetic,
    save_vector_field_animation,
)


In [ ]:
# Generování syntetického pole na mřížce (větší oblast, bez dlouhodobě preferovaného směru)
L = 2_000_000.0  # m (2000 km => -1000 .. 1000 km)
nx, ny = 21, 21
DAY = 24 * 3600.0
n_days = 30

x, y, t_grid, W_grid = generate_synthetic_w_grid(
    L=L,
    nx=nx,
    ny=ny,
    n_days=n_days,
    day=DAY,
)


In [ ]:
# Hodinová animace vektorového pole uložená do mp4
# Upozornění: generování animace může trvat delší dobu.

from IPython.display import Video, display

out_path = save_vector_field_animation(
    W_grid,
    x,
    y,
    t_grid,
    out_path="vector_field_hourly.mp4",
    day=DAY,
)

display(Video(out_path, embed=True))


In [ ]:
# Trajektorie v časově proměnném poli
def w_ext_grid(r, t):
    """
    Rychlost prostředí w z analytického syntetického pole.

    Poznámka: používáme analytickou funkci `w_synthetic`, takže v bodě (x, y) dostaneme
    přesnou hodnotu pole (nejde jen o vzorkování z mřížky).
    """
    return w_synthetic(r[0], r[1], t, L=L, day=DAY)


# Parametry simulace (odděleno od předchozích úkolů)
r0_field = np.array([0.0, 0.0])  # start uprostřed oblasti
v0_field = np.array([0.0, 0.0])

m_field = 0.001  # kg
alpha_field = 2.0e-5  # kg/m
kappa_field = alpha_field / m_field  # 1/m

T_field = 7.0 * DAY  # 7 dní

dt_sim = 10.0  # s (Euler je explicitní; pro toto kappa je 10 s rozumně stabilní)

t_field = np.arange(0.0, T_field + dt_sim, dt_sim)


def a_drift_cas(v, r, t):
    """Zrychlení pro drift v časově proměnném poli (kvadratický odpor)."""

    ###### doplňte kód zde ######
    v_rel = v - w_ext_grid(r, t)
    v_rel_norm = np.linalg.norm(v_rel)
    a = -kappa_field * v_rel_norm * v_rel
    #############################

    return a


r_td, v_td, a_td = simuluj_drift(r0_field, v0_field, dt_sim, T_field, a_drift_cas)

# Kontrola: typické rychlosti v poli by měly být v řádu jednotek m/s
speed = np.linalg.norm(v_td, axis=1)
print(f"Max. rychlost částice: {speed.max():.2f} m/s")

# Aby se grafy kreslily rychle i pro dlouhou simulaci, mírně je zředíme
stride_plot = max(1, t_field.size // 5000)  # cca do 5000 bodů v grafech
vykresli_drift(
    t_field[::stride_plot],
    r_td[::stride_plot],
    v_td[::stride_plot],
    title="Úkol 5: drift v časově proměnném poli",
)


---


## Úkol 6: Vliv odporu na trajektorii (realistické sady parametrů)

**Zadání:** Spočtěte trajektorie ze startu `r0_field` po dobu několika dnů (pro rychlost výpočtu)
pro několik realistických kombinací hmotnosti `m` a koeficientu odporu `alpha`
(a tedy `kappa = alpha / m`). Všechny trajektorie vykreslete do jednoho grafu

Použijte například tyto sady parametrů:
- lehké pěnové trosky: $m = 0.05\,\mathrm{kg}$, $\alpha = 1\times 10^{-3}\,\mathrm{kg}\,\mathrm{m}^{-1}$
- plastový kus: $m = 0.5\,\mathrm{kg}$, $\alpha = 1\times 10^{-3}\,\mathrm{kg}\,\mathrm{m}^{-1}$
- dřevěný kus: $m = 5.0\,\mathrm{kg}$, $\alpha = 1\times 10^{-3}\,\mathrm{kg}\,\mathrm{m}^{-1}$
- kovový kus: $m = 50.0\,\mathrm{kg}$, $\alpha = 1\times 10^{-3}\,\mathrm{kg}\,\mathrm{m}^{-1}$


*Poznámka:* V ukázkovém řešení počítáme 2 dny, aby se notebook spouštěl rychle. Chcete-li, můžete v kódu snadno změnit délku na 7 dní (např. použít `t_field`).


In [ ]:
# Úkol 6 – porovnání různých realistických sad parametrů pro stejný start a stejný čas
# Poznámka: pokud byste dali příliš velké alpha (nebo příliš malé m), kappa bude velké a
# explicitní Euler může při velkém dt_sim začít být nestabilní. Pak pomůže zmenšit dt_sim.

###### doplňte kód zde ######
parametry = [
    {"label": "lehké pěnové trosky", "m": 0.05, "alpha": 1.0e-3},
    {"label": "plastový kus", "m": 0.5, "alpha": 1.0e-3},
    {"label": "dřevěný kus", "m": 5.0, "alpha": 1.0e-3},
    {"label": "kovový kus", "m": 50.0, "alpha": 1.0e-3},
]
#############################

r0_cmp = np.array([10_000.0, 0.0])  # start posunutý, ať je vidět vliv prostorové variability pole

# pro porovnání si vezmeme kratší interval (2 dny) – kvůli rychlosti výpočtu
T_cmp = 30.0 * DAY
t_cmp = np.arange(0.0, T_cmp + dt_sim, dt_sim)

fig, ax = plt.subplots(figsize=(6, 6))

stride_plot = max(1, t_cmp.size // 5000)

for params in parametry:
    kappa_cmp = params["alpha"] / params["m"]
    label = f"{params['label']} (kappa={kappa_cmp:.3f} 1/m)"

    ###### doplňte kód zde ######
    # 1) definujte a_drift_cmp pro dané kappa_cmp
    # 2) spočtěte trajektorii
    # 3) vykreslete do společného grafu (v km)
    def a_drift_cmp(v, r, t, kappa=kappa_cmp):
        v_rel = v - w_ext_grid(r, t)
        v_rel_norm = np.linalg.norm(v_rel)
        return -kappa * v_rel_norm * v_rel

    r_cmp, v_cmp, _ = simuluj_drift(r0_cmp, v0_field, dt_sim, T_cmp, a_drift_cmp)
    ax.plot(r_cmp[::stride_plot, 0] / 1000.0, r_cmp[::stride_plot, 1] / 1000.0, label=label)
    #############################

ax.scatter(r0_cmp[0] / 1000.0, r0_cmp[1] / 1000.0, color="black", s=20, label="start")
ax.set_xlabel("x [km]")
ax.set_ylabel("y [km]")
ax.set_title("Úkol 6: vliv koeficientu odporu kappa na trajektorii (časově proměnné pole)")
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
ax.legend(loc="best", fontsize=8)
plt.tight_layout()
plt.show()


---


## 6. Shrnutí

**Hlavní body:**
- Umím formálně popsat $\mathbf{r}(t)$, $\mathbf{v}(t)$ a $\mathbf{a}(t)$ v 2D.
- Chápu, že jediná síla je odpor závislý na relativní rychlosti $\mathbf{v} - \mathbf{w}$.
- Vím, proč kombinujeme Eulerův krok pro $\mathbf{v}$ a lichoběžník pro $\mathbf{r}$.
- Umím volit rozumný krok $\Delta t$ a vím, že příliš velký krok zhorší přesnost.

**Doplňující poznámky (pro jistotu):**
- Rozumím významu koeficientu odporu $\kappa = \alpha/m$ a pole $\mathbf{w}$ a jejich vlivu na přizpůsobení rychlosti.
- Typické chyby: špatné jednotky, zapomenutá prealokace, pole špatného tvaru $(n, 2)$.
